# Nuclear Fusion Tokamak
![](./nuclearfusion_banner.png)

### Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, Vector3

be = BioExplorer('localhost:5000')
core = be.core_api()
be.reset_scene()
print('BioExplorer version ' + be.version())

### Generate random data set:
Vectors composed of a position and a direction. The length of the vector is the value of the voltage

In [ ]:
import math
import random
import numpy as np
import time

''' Settings '''
nb_particles = 2000
use_randomness = True

def generate_random_point_in_torus(main_radius, tube_radius):
    theta = 2 * math.pi * random.random()
    phi = 2 * math.pi * random.random()
    x = (main_radius + tube_radius * math.cos(phi)) * math.cos(theta)
    y = tube_radius * math.sin(phi)
    z = (main_radius + tube_radius * math.cos(phi)) * math.sin(theta)
    return x, y, z

origins = list()
targets = list()
origins_radii = list()
targets_radii = list()

seed = int(time.time())
random.seed(seed)
for i in range(nb_particles):
    x = 0
    y = 0
    z = 0
    x, y, z = generate_random_point_in_torus(10.0, 2.0)
    origin = Vector3(x, y ,z)
    

    length = 0.2
    cp = np.cross([x, y, z], [0, 1, 0])
    if use_randomness:
        cp[0] += (random.random() - 0.5)
        cp[1] += (random.random() - 0.5)
        cp[2] += (random.random() - 0.5)
        length = random.random() * 0.2

    target = Vector3(
        origin.x + cp[0] * length,
        origin.y + cp[1] * length,
        origin.z + cp[2] * length
    )
    origins.append(origin)
    targets.append(target)
    origins_radii.append(0.1)
    targets_radii.append(0.0)

status = be.add_cones(
    name='Particles',
    origins=origins, origins_radii=origins_radii,
    targets=targets, targets_radii=targets_radii,
    color=Vector3(1.0, 1.0, 1.0), opacity=1.0)

## Generate new model containing the field

In [ ]:
be.build_fields(
    voxel_size=0.25, density=1.0,
    data_type=be.FIELD_DATA_TYPE_VECTOR
)

status = core.set_renderer(current='advanced')

### Volume settings

In [ ]:
status = core.set_volume_parameters(
    pre_integration=False,
    specular=(1.0, 1.0, 1.0),
    user_parameters=(1.0, 100.0, 0.0), # second value (100.0) is the cutoff distance
    gradient_shading=False, single_shade=False, gradient_offset=5.0,
    sampling_rate=1.0, adaptive_sampling=False,
    adaptive_max_sampling_rate=1.0)

### Attach transfer function to newly created model

In [ ]:
from bioexplorer import TransferFunction
model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = TransferFunction(
    bioexplorer=be, model_id=model_id,
    filename='fusion.1dt',
    value_range=[0.1, 5.0])

## Fusion reactor Tokamak 3D model
Can be downloaded from: https://blendermarket.com/products/fusion-reactor-tokamak

In [ ]:
import os
transformation={
    'rotation': [0.0, -0.25038000405444144, 0.0, 0.9681476403781076],
    'rotation_center': [0.0, 0.0, 0.0],
    'scale': [12.0, 12.0, 12.0],
    'translation': [0.0, -1.0, 0.0]
}
model_filename = '/home/favreau/code/favreau/Notebooks/BioExplorerUseCases/NuclearFusion/data/Fusion_reactor_by_www_up3d_de_without_plasma.obj'
core.add_model(name='Fusion Reactor Tokamak', path=model_filename)
model_ids = be.get_model_ids()['ids']
status = core.update_model(id=model_ids[len(model_ids)-1], transformation=transformation)

## Rendering settings

In [ ]:
status = be.reset_camera()

In [ ]:
status = core.set_renderer(
    background_color=[0.25, 0.25, 0.25],
    current='advanced', 
    samples_per_pixel=1,
    subsampling=4,
    max_accum_frames=1024)
params = core.AdvancedRendererParams()
params.gi_samples = 0
params.gi_strength = 0.0
params.gi_ray_length = 150.0
params.shadow_intensity = 0.5
params.soft_shadow_strength = 0.1
params.max_ray_depth = 3
params.show_background = True
params.fog_start = 100.0
params.fog_thickness = 20.0
status = core.set_renderer_params(params)
status = core.set_renderer()

## Widgets to modify materials

In [ ]:
from bioexplorer import Widgets
w = Widgets(be)
w.display_palette_for_models()